## Observations

observation construct when there are $x$ number of adversaries, $y$ number of good_agents and $z$ number of landmarks:\
adversary $= 4+2z+2(x-1)+4y$\
good_agent $= 4+2z+2x+4(y-1)$

### 1. Environment Setup and Imports

In [40]:
# Import standard libraries
import numpy as np
import random
from collections import deque
from copy import deepcopy

# Import PyTorch and related libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Import PyTorch geometric libraries for GNNs
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv

# Import PettingZoo and Simple Tag environment
from pettingzoo.mpe import simple_tag_v3

# Import gym for action space
import gym

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


### 2. Modules for simple_tag_v2

##### 2.1 Observation wrapper

In [41]:
class ObservationWrapper:
    def __init__(self, env, num_class_A, num_class_B):
        self.env = env
        self.num_class_A = num_class_A
        self.num_class_B = num_class_B
        self.adversaries = [agent for agent in env.agents if 'adversary' in agent]
        self.good_agents = [agent for agent in env.agents if 'agent' in agent]
        self.landmarks = self._get_landmarks()

        # Assign classes to adversaries
        self.class_A_adversaries = self.adversaries[:num_class_A]
        self.class_B_adversaries = self.adversaries[num_class_A:num_class_A+num_class_B]
        self.adversary_classes = {}
        for adv in self.class_A_adversaries:
            self.adversary_classes[adv] = 0  # Class A
        for adv in self.class_B_adversaries:
            self.adversary_classes[adv] = 1  # Class B

    def _get_landmarks(self):
        # Assuming landmarks are part of the environment's state
        # Placeholder implementation
        return []

    def get_modified_observation(self, agent_name, original_observation):
        if agent_name in self.adversary_classes:
            class_id = self.adversary_classes[agent_name]
            if class_id == 0:
                # Class A adversary observation
                obs = self._get_class_A_observation(original_observation)
            else:
                # Class B adversary observation
                obs = self._get_class_B_observation(original_observation)
            # Add one-hot class ID
            class_id_one_hot = np.zeros(2)
            class_id_one_hot[class_id] = 1
            obs = np.concatenate([obs, class_id_one_hot])
            return obs
        else:
            # Return original observation for other agents
            return original_observation

    def _get_class_A_observation(self, original_observation):
        # Extract position and velocity of itself
        self_pos = original_observation[0:2]
        self_vel = original_observation[2:4]
        # Absolute positions of landmarks
        landmarks_pos = self._get_landmarks_positions()
        # Combine into a single observation
        obs = np.concatenate([self_pos, self_vel, landmarks_pos])
        return obs

    def _get_class_B_observation(self, original_observation):
        # Extract position and velocity of itself
        self_pos = original_observation[0:2]
        self_vel = original_observation[2:4]
        # Absolute position and velocity of the good agent
        good_agent_obs = self._get_good_agent_info()
        # Combine into a single observation
        obs = np.concatenate([self_pos, self_vel, good_agent_obs])
        return obs

    def _get_landmarks_positions(self):
        # Placeholder implementation
        # Return concatenated positions of landmarks
        landmarks_pos = []
        for lm in self.landmarks:
            pos = lm.state.p_pos  # Assuming landmark has position attribute
            landmarks_pos.extend(pos)
        return np.array(landmarks_pos)

    def _get_good_agent_info(self):
        # Assuming only one good agent for simplicity
        good_agent = self.good_agents[0]
        obs = self.env.observe(good_agent)
        pos = obs[0:2]
        vel = obs[2:4]
        return np.concatenate([pos, vel])

    def step(self, action_dict):
        # Apply actions and get new observations
        obs, rewards, dones, infos = self.env.step(action_dict)
        modified_obs = {}
        for agent_name, original_obs in obs.items():
            modified_obs[agent_name] = self.get_modified_observation(agent_name, original_obs)
        return modified_obs, rewards, dones, infos

    def reset(self):
        self.env.reset()


##### 2.2 Communication Mechanism  based on Eucledian distance

In [42]:
def get_neighbors_info(adversary_name, adversary_positions, communication_range):
    # Get the position of the current adversary
    current_pos = adversary_positions[adversary_name]
    neighbors = []
    for other_name, other_pos in adversary_positions.items():
        if other_name != adversary_name:
            distance = np.linalg.norm(current_pos - other_pos)
            if distance <= communication_range:
                neighbors.append(other_name)
    return neighbors


##### 2.3Construction of Graph for Commuincation (Testing)

In [43]:
def construct_dynamic_graph(adversary_positions, communication_range):
    nodes = []
    edge_index = [[], []]
    node_features = []
    node_mapping = {}
    
    # Assign indices to agents
    for idx, (adv_name, pos) in enumerate(adversary_positions.items()):
        node_mapping[adv_name] = idx
        nodes.append(adv_name)
        # Create node features (e.g., processed observations)
        feature = adversary_agents[adv_name].temporal_transformer.input_projection(adversary_agents[adv_name].temporal_transformer.input_projection(adversary_agents[adv_name].temporal_transformer.input_projection(processed_observations[adv_name]).detach()))
        node_features.append(feature)
    
    # Determine edges based on communication range
    for adv_i in nodes:
        idx_i = node_mapping[adv_i]
        pos_i = adversary_positions[adv_i]
        for adv_j in nodes:
            if adv_i != adv_j:
                pos_j = adversary_positions[adv_j]
                distance = np.linalg.norm(pos_i - pos_j)
                if distance <= communication_range:
                    idx_j = node_mapping[adv_j]
                    # Add edge from adv_i to adv_j
                    edge_index[0].append(idx_i)
                    edge_index[1].append(idx_j)
    
    # Convert to tensors
    if len(edge_index[0]) == 0:
        # No edges, use self-loop to prevent errors
        edge_index = torch.tensor([[0], [0]], dtype=torch.long).to(device)
    else:
        edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)
    node_features = torch.stack(node_features).to(device)
    
    return node_features, edge_index

##### 2.4 Init simple_tag_v2 and wrapper

In [44]:
# Initialize the environment
env = simple_tag_v3.env()
#env = simple_tag_v3.parallel_env()
env.reset()

# Define the number of adversaries in each class
num_class_A = 2
num_class_B = 2

# Wrap the environment
wrapper = ObservationWrapper(env, num_class_A=num_class_A, num_class_B=num_class_B)


### 3.Agent Networks

##### 3.1 Actor Network

In [45]:
class ActorNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ActorNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        action = torch.tanh(self.output_layer(x))  # Assuming continuous action space
        return action


##### 3.2. Critic Network

In [46]:
class CriticNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CriticNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, state_action):
        x = F.relu(self.fc1(state_action))
        x = F.relu(self.fc2(x))
        value = self.output_layer(x)
        return value


##### 3.3 Temporal Transformer

In [47]:
class TemporalTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers):
        super(TemporalTransformer, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.input_projection = nn.Linear(input_dim, hidden_dim)

    def forward(self, neighbor_sequences):
        # neighbor_sequences shape: (sequence_length, batch_size, input_dim)
        x = self.input_projection(neighbor_sequences)
        output = self.transformer(x)
        return output[-1]  # Return the last output (could be modified as needed)


##### 3.4 GNN Processor (Using GAT)

In [48]:
class GNNProcessor(nn.Module):
    def __init__(self, node_input_dim, hidden_dim, output_dim):
        super(GNNProcessor, self).__init__()
        self.conv1 = GATConv(node_input_dim, hidden_dim, heads=1)
        self.conv2 = GATConv(hidden_dim, output_dim, heads=1)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x


##### 3.5 GCN

In [49]:
class GCNProcessor(nn.Module):
    def __init__(self, node_input_dim, hidden_dim, output_dim):
        super(GCNProcessor, self).__init__()
        self.conv1 = GCNConv(node_input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # x: Node features, edge_index: Graph connectivity
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x


#### 3.6 Adversary Network

In [50]:
class AdversaryAgent:
    def __init__(self, name, class_id, observation_dim, action_dim, hidden_dim, transformer_params, actor, device):
        self.name = name
        self.class_id = class_id
        self.device = device
        self.observation_dim = observation_dim
        self.action_dim = action_dim
        self.hidden_dim = hidden_dim

        # Actor network (shared among class)
        self.actor = actor  # Shared actor passed during initialization

        # Temporal Transformer
        self.temporal_transformer = TemporalTransformer(
            input_dim=observation_dim,
            hidden_dim=hidden_dim,
            num_heads=transformer_params['num_heads'],
            num_layers=transformer_params['num_layers']
        ).to(device)

        # GNN Processor
        self.gnn_processor = GNNProcessor(
            node_input_dim=hidden_dim * 2,  # Transformer output + own processed obs
            hidden_dim=hidden_dim,
            output_dim=hidden_dim
        ).to(device)

        # Optimizers (if needed)
        self.optimizer = optim.Adam(self.parameters(), lr=1e-3)
    
    def parameters(self):
        return list(self.temporal_transformer.parameters()) + list(self.gnn_processor.parameters())
    
    def select_action(self, processed_obs, neighbor_info, edge_index):
        # processed_obs: Tensor of own processed observation
        # neighbor_info: Tensor of neighbor information from Temporal Transformer
        # edge_index: Edge connections for GNN

        # Combine own observation and neighbor info
        node_features = torch.cat([processed_obs, neighbor_info], dim=-1).unsqueeze(0)  # Add batch dimension

        # GNN processing
        x = node_features
        edge_index = edge_index.to(self.device)
        gnn_output = self.gnn_processor(x, edge_index)

        # Actor output
        action_probs = self.actor(gnn_output)
        action_distribution = torch.distributions.Categorical(action_probs)
        action = action_distribution.sample()
        return action.item(), action_distribution.log_prob(action)


### 4. Init Agents and Networks

In [60]:
# Get observation dimension after processing
sample_obs = wrapper.env.observe(wrapper.adversaries[0])
modified_obs = wrapper.get_modified_observation(wrapper.adversaries[0], sample_obs)
observation_dim = len(modified_obs)

# Get action dimension
action_space = env.action_space(wrapper.adversaries[0])
print(f'action_space: {action_space}')

action_dim = action_space.n


# Hyperparameters
hidden_dim = 128
transformer_params = {'num_heads': 2, 'num_layers': 2}

# Initialize class-specific actors
class_actors = {}
for class_id in [0, 1]:
    actor = ActorNetwork(input_dim=hidden_dim, hidden_dim=hidden_dim, output_dim=action_dim).to(device)
    actor.optimizer = optim.Adam(actor.parameters(), lr=1e-3)
    class_actors[class_id] = actor

# Initialize agents
adversary_agents = {}
for adv_name in wrapper.adversaries:
    class_id = wrapper.adversary_classes[adv_name]
    agent = AdversaryAgent(
        name=adv_name,
        class_id=class_id,
        observation_dim=observation_dim,
        action_dim=action_dim,
        hidden_dim=hidden_dim,
        transformer_params=transformer_params,
        actor=class_actors[class_id],
        device=device
    )
    adversary_agents[adv_name] = agent


action_space: Discrete(5)


### 5. Init Critic

In [62]:
use_class_based_critic = True

# Define global state dimension
global_state_dim = observation_dim * len(wrapper.adversaries) + observation_dim * len(wrapper.good_agents)

if use_class_based_critic:
    class_critics = {}
    for class_id in [0, 1]:
        critic = CriticNetwork(input_dim=global_state_dim, output_dim=1, hidden_dim=hidden_dim).to(device)
        critic.optimizer = optim.Adam(critic.parameters(), lr=1e-3)
        class_critics[class_id] = critic
else:
    centralized_critic = CriticNetwork(input_dim=global_state_dim, hidden_dim=hidden_dim).to(device)
    centralized_critic.optimizer = optim.Adam(centralized_critic.parameters(), lr=1e-3)


### 6. Training Loop

In [65]:
num_episodes = 1000
max_steps = 100
gamma = 0.99

for episode in range(num_episodes):
    wrapper.reset()
    observations = {agent: wrapper.env.observe(agent) for agent in wrapper.env.agents}
    done = False
    step = 0

    # Initialize episode memory for training
    episode_memory = []

    while not done and step < max_steps:
        actions = {}
        log_probs = {}
        rewards = {}
        # Process observations
        processed_observations = {}
        adversary_positions = {}
        for adv_name, agent in adversary_agents.items():
            obs = observations[adv_name]
            # Get modified observation
            obs = wrapper.get_modified_observation(adv_name, obs)
            obs_tensor = torch.tensor(obs, dtype=torch.float32).to(device)
            processed_observations[adv_name] = obs_tensor
            # Get position for communication
            adversary_positions[adv_name] = obs[0:2]

        # Get neighbor information and select actions
        for adv_name, agent in adversary_agents.items():
            # Get neighbors within communication range
            neighbors = get_neighbors_info(adv_name, adversary_positions, communication_range=1.0)
            print(f'for {adv_name}, neighbors: {neighbors} and their obs: {[observations[n] for n in neighbors]}')
            # Prepare neighbor sequences
            neighbor_sequences = []
            for neighbor in neighbors:
                neighbor_obs = processed_observations[neighbor]
                neighbor_sequences.append(neighbor_obs)
            if neighbor_sequences:
                neighbor_sequences = torch.stack(neighbor_sequences).unsqueeze(1).to(device)  # (seq_len, batch_size=1, input_dim)
                neighbor_sequences = neighbor_sequences.transpose(0, 1)  # Transformer expects (seq_len, batch_size, input_dim)
                neighbor_info = agent.temporal_transformer(neighbor_sequences)
            else:
                # If no neighbors, use zeros
                neighbor_info = torch.zeros(agent.hidden_dim).to(device)
            # Edge index for GNN (self-loop)
            edge_index = torch.tensor([[0], [0]], dtype=torch.long)
            # Select action
            action, log_prob = agent.select_action(processed_observations[adv_name], neighbor_info, edge_index)
            actions[adv_name] = action
            log_probs[adv_name] = log_prob

        # Assign random actions to good agents
        for good_agent in wrapper.good_agents:
            actions[good_agent] = env.action_space(good_agent).sample()

        # Step the environment
        observations_next, rewards, dones, infos = wrapper.step(actions)
        # Store experience
        for adv_name, agent in adversary_agents.items():
            reward = rewards[adv_name]
            done_flag = dones[adv_name]
            episode_memory.append({
                'agent_name': adv_name,
                'log_prob': log_probs[adv_name],
                'reward': reward,
                'done': done_flag
            })

        observations = observations_next
        done = all(dones.values())
        step += 1

    # After episode ends, compute returns and update networks
    # For simplicity, we'll use REINFORCE algorithm here
    returns = []
    R = 0
    for t in reversed(range(len(episode_memory))):
        R = episode_memory[t]['reward'] + gamma * R
        returns.insert(0, R)

    # Normalize returns
    returns = torch.tensor(returns, dtype=torch.float32).to(device)
    returns = (returns - returns.mean()) / (returns.std() + 1e-9)

    # Update actors
    policy_loss = {}
    for idx, data in enumerate(episode_memory):
        adv_name = data['agent_name']
        agent = adversary_agents[adv_name]
        class_id = agent.class_id
        log_prob = data['log_prob']
        R = returns[idx]
        loss = -log_prob * R
        if class_id not in policy_loss:
            policy_loss[class_id] = loss
        else:
            policy_loss[class_id] += loss

    for class_id, loss in policy_loss.items():
        class_actors[class_id].optimizer.zero_grad()
        loss.backward()
        class_actors[class_id].optimizer.step()

    # Update critics if using class-based critic
    if use_class_based_critic:
        # Prepare global state
        global_state = []
        for adv_name in wrapper.adversaries:
            obs = processed_observations[adv_name].detach().cpu().numpy()
            global_state.extend(obs)
        for good_agent in wrapper.good_agents:
            obs = wrapper.env.observe(good_agent)
            global_state.extend(obs)
        global_state = torch.tensor(global_state, dtype=torch.float32).to(device)

        # Compute value targets and update critics
        for class_id, critic in class_critics.items():
            value = critic(global_state)
            target = torch.tensor([R], dtype=torch.float32).to(device)
            critic_loss = F.mse_loss(value, target)
            critic.optimizer.zero_grad()
            critic_loss.backward()
            critic.optimizer.step()
    else:
        # Update centralized critic
        global_state = []
        for adv_name in wrapper.adversaries:
            obs = processed_observations[adv_name].detach().cpu().numpy()
            global_state.extend(obs)
        for good_agent in wrapper.good_agents:
            obs = wrapper.env.observe(good_agent)
            global_state.extend(obs)
        global_state = torch.tensor(global_state, dtype=torch.float32).to(device)

        value = centralized_critic(global_state)
        target = torch.tensor([R], dtype=torch.float32).to(device)
        critic_loss = F.mse_loss(value, target)
        centralized_critic.optimizer.zero_grad()
        critic_loss.backward()
        centralized_critic.optimizer.step()

    if episode % 10 == 0:
        print(f"Episode {episode} completed")


for adversary_0, neighbors: ['adversary_1', 'adversary_2'] and their obs: [array([ 0.        ,  0.        , -0.0270679 , -0.8876596 , -0.089927  ,
        1.3559995 ,  0.3937757 ,  0.07579712, -0.04470859,  0.3539868 ,
        0.92530453,  1.2470239 , -0.32683226,  0.5439994 ,  0.        ,
        0.        ], dtype=float32), array([ 0.        ,  0.        ,  0.89823663,  0.35936433, -1.0152315 ,
        0.10897546, -0.53152883, -1.1712269 , -0.9700131 , -0.89303714,
       -0.92530453, -1.2470239 , -1.2521368 , -0.70302457,  0.        ,
        0.        ], dtype=float32)]


RuntimeError: stack expects each tensor to be equal size, but got [6] at entry 0 and [10] at entry 1

### 7.Save Models

In [ ]:
# Save models
for class_id, actor_info in class_actors.items():
    torch.save(actor_info['network'].state_dict(), f'class_{class_id}_actor.pth')

#if use_class_based_critic:
#    for class_id, critic_info in class_critics.items():
#        torch.save(critic_info['network'].state_dict(), f'class_{class_id}_critic.pth')
#else:
#    torch.save(centralized_critic.state_dict(), 'centralized_critic.pth')


### 8.Testing Function

In [ ]:
def test_model(env, wrapper, adversary_agents, class_actors, class_critics, use_class_based_critic, num_class_A, num_class_B, num_test_episodes=10, max_test_steps=1000):
    """
    Tests the trained model on a different configuration of adversaries.

    Parameters:
    - env: PettingZoo environment instance.
    - wrapper: ObservationWrapper instance.
    - adversary_agents: Dictionary of AdversaryAgent instances.
    - class_actors: Dictionary of class actors.
    - class_critics: Dictionary of class critics.
    - use_class_based_critic: Boolean flag for critic type.
    - num_class_A: Number of adversaries in Class A for testing.
    - num_class_B: Number of adversaries in Class B for testing.
    - num_test_episodes: Number of test episodes.
    - max_test_steps: Maximum steps per episode.

    Returns:
    - average_reward: Average reward over test episodes.
    """
    total_rewards = []

    for episode in range(num_test_episodes):
        # Reset the environment and wrapper with new class configurations
        wrapper.num_class_A = num_class_A
        wrapper.num_class_B = num_class_B
        modified_obs = wrapper.reset()
        done = False
        step = 0
        episode_reward = 0

        while not done and step < max_test_steps:
            actions = {}
            # Process observations and select actions for each adversary
            adversary_positions = {}
            processed_observations = {}
            for adv_name, agent in adversary_agents.items():
                if adv_name not in wrapper.adversaries:
                    continue  # Skip adversaries not present in this test
                obs = modified_obs[adv_name]
                # Get processed observation
                obs_tensor = torch.tensor(obs, dtype=torch.float32).to(device)
                processed_observations[adv_name] = obs_tensor
                # Get position for communication (assuming first two elements are position)
                adversary_positions[adv_name] = obs[0:2]

            # Communication and neighbor info
            neighbor_infos = {}
            edge_indices = {}
            for adv_name, agent in adversary_agents.items():
                if adv_name not in wrapper.adversaries:
                    continue  # Skip adversaries not present in this test
                # Get neighbors within communication range
                neighbors = get_neighbors_info(adv_name, adversary_positions, communication_range=1.0)
                # Prepare neighbor sequences
                neighbor_sequences = []
                for neighbor in neighbors:
                    neighbor_obs = processed_observations[neighbor]
                    neighbor_sequences.append(neighbor_obs)
                if neighbor_sequences:
                    neighbor_sequences = torch.stack(neighbor_sequences).unsqueeze(1).to(device)  # (seq_len, batch_size=1, input_dim)
                    neighbor_info = agent.temporal_transformer(neighbor_sequences)
                else:
                    # If no neighbors, use zeros
                    neighbor_info = torch.zeros(agent.hidden_dim).to(device)
                neighbor_infos[adv_name] = neighbor_info

                # Construct dynamic graph
                node_features, edge_index = construct_dynamic_graph(adversary_positions, communication_range=1.0)
                edge_indices[adv_name] = edge_index

            # Select actions
            for adv_name, agent in adversary_agents.items():
                if adv_name not in wrapper.adversaries:
                    continue  # Skip adversaries not present in this test
                action, log_prob = agent.select_action(
                    processed_observations[adv_name], 
                    neighbor_infos[adv_name], 
                    edge_indices[adv_name]
                )
                actions[adv_name] = action

            # Assign random actions to good agents
            for good_agent in wrapper.good_agents:
                actions[good_agent] = env.action_space(good_agent).sample()

            # Step the environment
            modified_obs_next, rewards, dones, infos = wrapper.step(actions)

            # Accumulate rewards
            for adv_name in wrapper.adversaries:
                episode_reward += rewards[adv_name]

            modified_obs = modified_obs_next
            done = all(dones.values())
            step += 1

        total_rewards.append(episode_reward)
        print(f"Test Episode {episode + 1}/{num_test_episodes} Reward: {episode_reward}")

    average_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_test_episodes} Test Episodes: {average_reward}")
    return average_reward


### 9.Test on different number of agents

In [2]:
# To test on different configurations, reinitialize the environment and agents
test_num_class_A = 3
test_num_class_B = 1

# Create new environment and wrapper for testing
test_env = simple_tag_v3.parallel_env()
test_env.reset()
wrapper_test = ObservationWrapper(test_env, num_class_A=test_num_class_A, num_class_B=test_num_class_B)

# Load models
for class_id, actor in class_actors.items():
    actor.load_state_dict(torch.load(f'class_{class_id}_actor.pth'))

# Reinitialize agents for testing
adversary_agents_test = {}
for adv_name in wrapper_test.adversaries:
    class_id = wrapper_test.adversary_classes[adv_name]
    agent = AdversaryAgent(
        name=adv_name,
        class_id=class_id,
        observation_dim=observation_dim,
        action_dim=action_dim,
        hidden_dim=hidden_dim,
        transformer_params=transformer_params,
        actor=class_actors[class_id],
        device=device
    )
    adversary_agents_test[adv_name] = agent
    


NameError: name 'ObservationWrapper' is not defined